<a href="https://colab.research.google.com/github/quiqueacla/IntrinsecValue_dcf/blob/devel/Intrinsec_Value_DCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Programa para facilitar la busqueda de datos para completar el excel de calculo de valor inrinseco de una compañía.
Seleccionar el TICKER de la compañia en la primera celda. Y ejecutar.
El programa busca en Yahoo Finance los valores para poder hacer copy Paste en el excel y facilitar el rellenado.

Luego en el EXCEL jugar con los parámetros restantes.

TODO: Hacer un jupiternotbook que automaticamente calcule el valor intrinseco.
Pero de todos modos finalmente hay que jugar con ciertos parámetros, puesto que a veces las compañias no se pueden valorar con los mismos parámetros.

In [ ]:
!pip -q install yfinance pandas numpy
import locale
import yfinance as yf
import pandas as pd
import numpy as np



#pd.set_option("display.float_format", lambda x: f"{x:,.2f}")
pd.set_option(
    "display.float_format",
    lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
)

TICKER = "IDR.MC"   # <-- cambia aquí si quieres otro
#TICKER = "VOW.DE"   # <-- cambia aquí si quieres otro
#TICKER = "MBG.DE"   # <-- cambia aquí si quieres otro
t = yf.Ticker(TICKER)


#Get General Data
info = t.info or {}
SharePrice  = info.get("currentPrice", np.nan)
Beta        = info.get("beta", np.nan)
NumShares   = info.get("sharesOutstanding", np.nan)
MarketCap = NumShares*SharePrice
TotalLiabilities = info.get("totalDebt", np.nan)
TotalAssets = info.get("totalAssets", np.nan)
DebtToEquity = info.get("debtToEquity", np.nan)

# display(info)
#Datos para proyeccion FCFF

#Calculo de las Estadísticas pasadas
Se recuperan los datos de los 4 años pasados y se hacen las estadíticas de margenes sobre los datos de interés. Estos nos serán de utilidad para hacer las proyecciones.


In [ ]:
##################################
# Data from Finantial statements
#################################

#Function to reorder columns
def tidy(df):
    if df is None or df.empty:
        return df
    out = df.copy()
    out.columns = pd.to_datetime(out.columns).date
    # ordenar columnas cronológicamente (ascendente)
    out = out.reindex(sorted(out.columns), axis=1)
    return out

#Funcion to pick a row from Table
def pick_row(df, names):
    if df is None or df.empty:
        return None
    idx = set(df.index.astype(str))
    for n in names:
        if n in idx:
            s = df.loc[n].astype(float)
            s.index = pd.to_datetime(s.index).date
            return s
    return None

#Get Finantial Statements
inc = t.financials
bal = t.balance_sheet
cf  = t.cashflow

#Order Tables
inc = tidy(inc)
bal = tidy(bal)
cf  = tidy(cf )

#Date form Incoming statement
Revenues = pick_row(inc, ["Total Revenue", "Revenue"])
NetIncome = pick_row(inc, ["Net Income", "Net Income Common Stockholders"])
NetIncomMargin = NetIncome / Revenues
InterestExpense = pick_row(inc, ["Interest Expense"])

EBIT = pick_row(inc, ["Ebit", "EBIT", "Operating Income", "OperatingIncome"])

IncomeTaxExpense = pick_row(inc, ["Tax Provision","Income Tax Expense"])
IncomeBeforeTax  = pick_row(inc, ["Income Before Tax", "Pretax Income"])
TaxRate          = IncomeTaxExpense / IncomeBeforeTax
TaxRate_mean     = TaxRate.mean()

Capex = pick_row(bal, ["Capital Expenditures"])

#Data from Balance
TotalDebt = pick_row(bal, ["Total Debt"])

TotalCurrentAssets  = pick_row(bal, ["Total Current Assets", "Current Assets"])
TotalCurrentLiabilities = pick_row(bal, ["Total Current Liabilities", "Current Liabilities"])
DeltaWorkingCapital = TotalCurrentAssets - TotalCurrentLiabilities

#Data from CashFlow
CFO = pick_row(cf, ["Operating Cash Flow"])
DA = pick_row(cf, ["Depreciation And Amortization"])
Capex = pick_row(cf, ["Capital Expenditures", "Capital Expenditure"])
Capex = -Capex;

###############################################
# BUILD Table to calculate TaxRate and CostLoan
###############################################
CostLoan = InterestExpense / TotalDebt
df_CostLoan_TaxRate = pd.DataFrame({
    "TotalDebt" : TotalDebt,
    "InterestExpense" : InterestExpense,
    "CostLoan" : CostLoan,
    "EBIT":EBIT,
    "IncomeBeforeTax":IncomeBeforeTax,
    "IncomeTaxExpense":IncomeTaxExpense,
    "TaxRate" : TaxRate,
})
display(df_CostLoan_TaxRate.T)

CostLoan_mean = CostLoan.mean()
print("--COST OF DEBT PARS--")
print(f"CostLoan  : {CostLoan_mean*100:.2f} %")
print(f"TaxRate   : {TaxRate_mean*100:.2f} %")
print("--COST OF EQUITY PARS--")
print("Beta      : ", Beta)
print("--MAIN INFORMATION--")
print("SharePrice: ", SharePrice)
print("NumShares : ", NumShares)



print("MarketCap : ", MarketCap)
print("--")
print("TotalLiabilities: ", TotalLiabilities)
print("TotalAssets: ", TotalAssets)
print("%Equity: ", MarketCap/(MarketCap+TotalLiabilities))
print("%Debt: ", TotalLiabilities/(MarketCap+TotalLiabilities))


###############################################
# BUILD Table to calculate Projections
###############################################
#Calculate FCFF
FCFF = CFO - Capex + InterestExpense*(1-TaxRate_mean)
#FCFF_1 = EBIT*(1-TaxRate_mean) + DA - Capex - DeltaWorkingCapital

df_Proyecciones_0 = pd.DataFrame({
    "Revenues" : Revenues,
    "NetIncome" : NetIncome,
    "NetIncomMargin" : NetIncomMargin,
    "CFO" : CFO,
    "Capex" : Capex,
    "DA" : DA,
    "InterestExpense":InterestExpense,
    "FCFF":FCFF,
})

print("--Tabla Proyecciones: Pasadas--")
display(df_Proyecciones_0.T)

# df1 = pd.DataFrame({
#     "EBIT":EBIT,
#     "IncomeBeforeTax":IncomeBeforeTax,
#     "IncomeTaxExpense":IncomeTaxExpense,
#     "TaxRate":TaxRate,
#     "CFO":CFO,
#     "Capex":Capex,
#     "DA":DA,
#     "TotalCurrentAssets":TotalCurrentAssets,
#     "TotalCurrentLiabilities":TotalCurrentLiabilities,
#     "DeltaWorkingCapital":DeltaWorkingCapital,
# })
# display(df1.T)






##Cálculo del Intrinsec Value
En base a los resultados obtenidos en los pasos anteriores sobre datos históricos. Se realiza las proyecciones de:
 - Revenues: Los dos primeros años basado en la estimaciones de los analista de Yahho
 - Los siguientes proyectados por estadistica
 - Se proyectan los FCFF futuros.

Luego e realizan 3 escenarios trayendo a valor presente los FCFF proyectados y el terminal value.

Al final de todo se saca la tabla resumen de los tres escenarios con sus parámetros y el margen de seguridad. Así como la media de los tres.

__Importante__:  los WACC se calculan a partir de los balances, así como la Beta se casa de YF. Pero a veces hay que evaluar la conveniencia de usar estos, o usar WACC sectoriales o Betas más realistas de cara a hacer estimaciones.

In [ ]:
#############################################################
# Calculo de Proyecciones
# En base a los 4 años pasados, y las estimaciones de analistas
# para y0 y y1, proyectamos FFCF para y0,y1,y2,y3,y4
#############################################################
#PARS
TerminalGrowth_neutro = 0.02
WAC_neutro_typ = 0.09#WACC

RiskFreeRate = 0.029    #Bono soberano 10y
EquityRiskPrime = 0.07  # mercados EUR
CostEquity = RiskFreeRate + EquityRiskPrime * Beta

CostLoan = CostLoan_mean
CostDebt = CostLoan *(1-TaxRate_mean)

E_p = 1 / (DebtToEquity/100 + 1)
D_p = 1-E_p

#Seleccionar el WACC calculado de balance o el typico de parametro
#WAC_neutro = E_p * CostEquity + D_p * CostDebt
WAC_neutro = WAC_neutro_typ

#Revenues Estimadas de los analista para y0 y1
rev_est = t.revenue_estimate
re_y0 = rev_est.loc["0y","avg"]
re_y1 = rev_est.loc["+1y","avg"]

#Generamos el array de ventas pasadas y estimadas y sacamos el Growth
Revenues = pick_row(inc, ["Total Revenue", "Revenue"])
Revenues = np.array(Revenues)
Revenues_next = np.array([re_y0, re_y1])
Revenues = np.concatenate([Revenues, Revenues_next])

Growth = (Revenues[1:] - Revenues[:-1]) / Revenues[:-1]
Growth_mean = Growth.mean()
Growth_std = Growth.std()
#Con el crecimiento medio estimamos los años siguientes y2 y3 y4
tmp = []
tmp.append(Revenues[-1]*(1+Growth_mean))
tmp.append(Revenues[-1]*(1+Growth_mean)**2)
tmp.append(Revenues[-1]*(1+Growth_mean)**3)
Revenues = np.concatenate([Revenues, tmp])
Growth = (Revenues[1:] - Revenues[:-1]) / Revenues[:-1]

#Calculamos el NetIncome y NetIncomMargin
#NetIncome = Revenues*NetIncomMargin
NetIncome = pick_row(inc, ["Net Income", "Net Income Common Stockholders"])

NetIncomeMargin = np.array(NetIncome[0:4])/np.array(Revenues[0:4])
NetIncomeMargin_mean = NetIncomeMargin.mean()
NetIncomeMargin = np.append(NetIncomeMargin, [NetIncomeMargin_mean]*5)

NetIncome = Revenues * NetIncomeMargin



#CashFlow, Capex e Intereses para sacat FFCF
CFO = pick_row(cf, ["Operating Cash Flow"])
Capex = pick_row(cf, ["Capital Expenditures", "Capital Expenditure"])
InterestExpense = pick_row(inc, ["Interest Expense"])

CFO=np.array(CFO)
Capex=np.array(Capex)
Capex=-Capex
InterestExpense=np.array(InterestExpense)

FCF_gen = CFO - Capex
FCFF    = CFO - Capex + InterestExpense*(1-TaxRate_mean)

FCFF_NI_ratio = FCFF/NetIncome[:4]
FCFF_NI_ratio_mean = FCFF_NI_ratio.mean()
FCFF_NI_ratio = np.append(FCFF_NI_ratio, [FCFF_NI_ratio_mean]*5)
FCFF = NetIncome * FCFF_NI_ratio #Machacamos los 4 primeros pero no pasa nada porque da lo mismo

################################################################
# Presentación Parámetros y protecciones
################################################################
BOLD = "\033[1m"
RESET = "\033[0m"
print(f"\n{BOLD} PARÁMETROS COMUNES y PROYECCIONES     {RESET}")
print(f"{BOLD}-----------------------------------------{RESET}")
#CostEquity = RiskFreeRate + EquityRiskPrime * Beta
print(f"RiskFreeRate (Rf):\t{RiskFreeRate*100:.2f}% (from Program Constants)")
print(f"EquityPrime (EPR):\t{EquityRiskPrime*100:.2f}% (from Program Constants)")
print(f"Beta             :\t{Beta:.2f}")
print(f"CostEquity   (Ke):\t{CostEquity*100:.2f}%")

#CostDebt = CostLoan *(1-TaxRate_mean)
print(f"CostLoan     (Cl):\t{CostLoan*100:.2f}% (from IncomeStatement)")
print(f"TaxRate_mean  (T):\t{TaxRate_mean*100:.2f}% (from IncomeStatement)")
print(f"CostDebt     (Kd):\t{CostDebt*100:.2f}%")

print(f"Debt/Equity (D/E):\t{DebtToEquity:.2f}% (from yfinance variable)")
print(f"Equity/Assets (E):\t{E_p*100:.2f}%")
print(f"Debt/Assets   (D):\t{D_p*100:.2f}%")
print(f"{BOLD}WACC_neutro:\t{WAC_neutro*100:.2f}%{RESET} (Typ {WAC_neutro_typ*100:.2f}%)= (E/(E+D) x Ke + (D/(E+D) x Kd))")
print(f"{BOLD}Tg_neutro  :\t{TerminalGrowth_neutro*100:.2f}%{RESET}")

#Fill columns not used to print table with panda
CFO = np.append(CFO, [-1]*5)
Capex = np.append(Capex, [-1]*5)
InterestExpense = np.append(InterestExpense, [-1]*5)
FCF_gen = np.append(FCF_gen, [-1]*5)
Growth =np.append(Growth, [-1])

df_Proyecciones_1 = pd.DataFrame({
    "Revenues" : Revenues,
    "Growth" : Growth,
    "NetIncome" : NetIncome,
    "NetIncomMargin" : NetIncomeMargin,
    "CFO" : CFO,
    "Capex" : Capex,
    "FCFF_gen":FCF_gen,
    "InterestExpense":InterestExpense,
    "FCFF":FCFF,
})
df_Proyecciones_1_Transpuesta = df_Proyecciones_1.T
df_Proyecciones_1_Transpuesta.columns = ["-4y", "-3y", "-2y", "-1y", "y0", "+1y", "+2y", "+3y", "+4y"]
display(df_Proyecciones_1_Transpuesta)


#########################################################
# Calculo de Escenarios
#########################################################
scenarios = [
    {"name": "Bajista",  "delta_wacc": +0.02, "delta_tg": -0.01},
    {"name": "Neutro",   "delta_wacc":  0.00, "delta_tg":  0.000},
    {"name": "Alcista",  "delta_wacc": -0.02, "delta_tg": +0.01},
]

Summary={}

for s in scenarios:

    Tg   = TerminalGrowth_neutro + s["delta_tg"]
    WACC = WAC_neutro + s["delta_wacc"]

    # print(
    #     f"{s['name']:8s} | "
    #     f"WACC={WACC*100:.2f}% | "
    #     f"g={Tg*100:.2f}%"
    # )


    FCFF_plus_TerminalValue = FCFF[-5:-1] #Cogemos y0,y1,y2,y3 (y4 lo dejamos)
    TerminalValue = FCFF_plus_TerminalValue[-1]*(1+Tg)/(WACC-Tg)

    FCFF_plus_TerminalValue = np.append(FCFF_plus_TerminalValue, TerminalValue)
    DiscountFactor = 1/(1+WACC)**np.arange(1,6)
    DiscountFactor[-1] = DiscountFactor[-2]
    PresentValue = FCFF_plus_TerminalValue * DiscountFactor

    ######################################################
    # Presentacion
    ######################################################
    print(f"\n{BOLD}{'-' * 100}{RESET}")
    print(f"{BOLD} Análisis ESCENARIO {s['name'].upper()}{RESET}")
    print(f"{BOLD}{'-' * 100}{RESET}")
    print(f"{BOLD}WACC             :\t{WACC*100:.2f}%{RESET}= (E/(E+D) x Ke + (D/(E+D) x Kd))")
    print(f"{BOLD}Terminal Growth  :\t{Tg*100:.2f}%{RESET}")
    print()

    f"Scenario: {s['name']}"
    title = f"<b>FCFF para {TICKER}. Escenario Scenario: {s['name']}:\nWACC {WACC*100:.2f}%  Growth {Tg*100:.2f}</b>"

    df_FCFF_Neutro = pd.DataFrame({
        "FCFF_plus_TerminalValue":FCFF_plus_TerminalValue,
        "DiscountFactor":DiscountFactor,
        "PresentValue":PresentValue,
    })
    df_FCFF_Neutro_Transpuesta = df_FCFF_Neutro.T
    df_FCFF_Neutro_Transpuesta.columns = ["y0", "y1", "y2", "y3", "TerminalValue"]
    df_FCFF_Neutro_Transpuesta.style.set_caption(title)
    display(df_FCFF_Neutro_Transpuesta.style.set_caption(title))


    ValorIntrinsec_per_share = PresentValue.sum()/NumShares
    print("")
    print("")
    title = f"<b>Valor Intrinseco {TICKER}. Valor Intrinseco</b>"
    df_VI_Neutro = pd.DataFrame({
        "Valor intrinseco": [PresentValue.sum()],
        "SharesOut": [NumShares],
        "Valor intrinseco por accion": [ValorIntrinsec_per_share],
    })
    display(df_VI_Neutro.T.style.set_caption(title))


    Summary[s["name"]] = {
        "WACC":WACC,
        "Tg":Tg,
        "CurrentPrice":SharePrice,
        "IntrinsecValue":ValorIntrinsec_per_share,
        "Margin":(ValorIntrinsec_per_share/SharePrice - 1)
    }


print(f"\n\n")
df = pd.DataFrame(Summary)
df["Mean"] = df.mean(axis=1)

rows_percent = ["Tg", "WACC", "Margin"]
rows_perun   = ["CurrentPrice", "IntrinsecValue"]
title = f"<b>Escenarios DCF – {TICKER}</b>"

idx = pd.IndexSlice
display(
    df.style
    .format("{:.2%}", subset=idx[rows_percent, :])
    .format("{:,.2f} €", subset=idx[rows_perun, :])
    .set_caption(title)
)





